In [ ]:
import random

# Define constants
NUM_DAYS = 5
NUM_SLOTS_PER_DAY = 7
NUM_CLASSROOMS = 10
NUM_SECTIONS = 20
NUM_PROFESSORS = 15
NUM_COURSES = 30
MAX_COURSES_PER_PROF = 3
MAX_COURSES_PER_SECTION = 5
POPULATION_SIZE = 100
MAX_GENERATIONS = 1000
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8

# Define room sizes
ROOM_SIZES = [60, 120]

# Define time slots
TIME_SLOTS = [(day, slot) for day in range(NUM_DAYS) for slot in range(NUM_SLOTS_PER_DAY)]

# Define chromosome representation
def generate_chromosome():
    genes = []
    for course in range(NUM_COURSES):
        is_lab = random.random() < 0.3  # 30% chance of being a lab course
        section = random.randint(0, NUM_SECTIONS - 1)
        professor = random.randint(0, NUM_PROFESSORS - 1)
        room1 = random.randint(0, NUM_CLASSROOMS - 1)
        room2 = random.randint(0, NUM_CLASSROOMS - 1)
        time_slot1 = random.choice(TIME_SLOTS)
        if is_lab:
            time_slot2 = (time_slot1[0], time_slot1[1] + 2)  # Lab courses have 3 consecutive slots
        else:
            time_slot2 = random.choice([(day, slot) for day, slot in TIME_SLOTS if day != time_slot1[0] and abs(slot - time_slot1[1]) > 1])
        genes.append((course, is_lab, section, professor, room1, time_slot1, room2, time_slot2))
    return genes

# Define fitness function
def fitness(chromosome):
    # Implement hard constraints
    professor_courses = [[] for _ in range(NUM_PROFESSORS)]
    section_courses = [[] for _ in range(NUM_SECTIONS)]
    room_slots = [[] for _ in range(NUM_CLASSROOMS)]

    for gene in chromosome:
        course, is_lab, section, professor, room1, time_slot1, room2, time_slot2 = gene

        # Check if professor can teach the course
        if len(professor_courses[professor]) >= MAX_COURSES_PER_PROF:
            return float('inf')

        # Check if section can have the course
        if len(section_courses[section]) >= MAX_COURSES_PER_SECTION:
            return float('inf')

        # Check if room is available for time slots
        if time_slot1 in room_slots[room1] or time_slot2 in room_slots[room2]:
            return float('inf')

        # Check if room size is sufficient
        section_strength = len([course for course in section_courses[section] if not course[1]])
        if section_strength + (0 if is_lab else 1) > ROOM_SIZES[0] and (room1 not in [r for r, size in enumerate(ROOM_SIZES) if size == 120] or room2 not in [r for r, size in enumerate(ROOM_SIZES) if size == 120]):
            return float('inf')

        # Check if professor is available for time slots
        if time_slot1 in [slot for course in professor_courses[professor] for slot in course[5]] or time_slot2 in [slot for course in professor_courses[professor] for slot in course[5]]:
            return float('inf')

        # Check if section is available for time slots
        if time_slot1 in [slot for course in section_courses[section] for slot in course[5]] or time_slot2 in [slot for course in section_courses[section] for slot in course[5]]:
            return float('inf')

        # Update lists
        professor_courses[professor].append(gene)
        section_courses[section].append(gene)
        room_slots[room1].append(time_slot1)
        room_slots[room2].append(time_slot2)

    # Implement soft constraints
    score = 0

    # Minimize number of floors traversed
    floor_counts = {}
    for gene in chromosome:
        course, is_lab, section, professor, room1, time_slot1, room2, time_slot2 = gene
        floor1 = room1 // (NUM_CLASSROOMS // NUM_DAYS)
        floor2 = room2 // (NUM_CLASSROOMS // NUM_DAYS)
        floor_counts[professor] = floor_counts.get(professor, [])
        floor_counts[professor].append(floor1)
        floor_counts[professor].append(floor2)
        floor_counts[section] = floor_counts.get(section, [])
        floor_counts[section].append(floor1)
        floor_counts[section].append(floor2)
    for floors in floor_counts.values():
        score += len(set(floors)) - 1

    # Prefer longer continuous teaching blocks
    for gene in chromosome:
        course, is_lab, section, professor, room1, time_slot1, room2, time_slot2 = gene
        if not is_lab and time_slot1[0] == time_slot2[0] and time_slot1[1] + 1 == time_slot2[1]:
            score += 1

    # Prefer same classroom across the week
    for gene in chromosome:
        course, is_lab, section, professor, room1, time_slot1, room2, time_slot2 = gene
        if room1 != room2:
            score += 1

    # Prefer morning sessions for theory classes
    for gene in chromosome:
        course, is_lab, section, professor, room1, time_slot1, room2, time_slot2 = gene
        if not is_lab and (time_slot1[1] >= NUM_SLOTS_PER_DAY // 2 or time_slot2[1] >= NUM_SLOTS_PER_DAY // 2):
            score += 1

    # Prefer afternoon sessions for lab classes
    for gene in chromosome:
        course, is_lab, section, professor, room1, time_slot1, room2, time_slot2 = gene
        if is_lab and (time_slot1[1] < NUM_SLOTS_PER_DAY // 2 or time_slot2[1] < NUM_SLOTS_PER_DAY // 2):
            score += 1

    return score

# Define genetic operators
def selection(population):
    return random.sample(population, 2)

def crossover(parent1, parent2):
    # Perform single-point crossover
    crossover_point = random.randint(0, len(parent1))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(chromosome):
    # Mutate a random gene
    if random.random() < MUTATION_RATE:
        gene_idx = random.randint(0, len(chromosome) - 1)
        gene = chromosome[gene_idx]
        course, is_lab, section, professor, room1, time_slot1, room2, time_slot2 = gene

        # Mutate a random attribute of the gene
        mutation_choice = random.randint(0, 3)
        if mutation_choice == 0:
            # Mutate section
            section = random.randint(0, NUM_SECTIONS - 1)
        elif mutation_choice == 1:
            # Mutate professor
            professor = random.randint(0, NUM_PROFESSORS - 1)
        elif mutation_choice == 2:
            # Mutate rooms
            room1 = random.randint(0, NUM_CLASSROOMS - 1)
            room2 = random.randint(0, NUM_CLASSROOMS - 1)
        else:
            # Mutate time slots
            time_slot1 = random.choice(TIME_SLOTS)
            if is_lab:
                time_slot2 = (time_slot1[0], time_slot1[1] + 2)
            else:
                # Ensure time_slot2 is different day and not consecutive slot
                available_slots = [(day, slot) for day, slot in TIME_SLOTS if day != time_slot1[0] and abs(slot - time_slot1[1]) > 1]
                time_slot2 = random.choice(available_slots)

        # Create mutated gene
        mutated_gene = (course, is_lab, section, professor, room1, time_slot1, room2, time_slot2)
        chromosome[gene_idx] = mutated_gene

    return chromosome


# Genetic algorithm
def genetic_algorithm():
    population = [generate_chromosome() for _ in range(POPULATION_SIZE)]
    for generation in range(MAX_GENERATIONS):
        # Evaluate fitness
        population = sorted(population, key=lambda x: fitness(x))

        # Check termination condition
        best_chromosome = population[0]
        if fitness(best_chromosome) == 0:
            break

        # Create new population
        new_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = selection(population)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1))
            new_population.append(mutate(child2))

        population = new_population

    return population[0]

# calling the genetic function
best_chromosome = genetic_algorithm()
print("Best solution found:")
for gene in best_chromosome:
    course, is_lab, section, professor, room1, time_slot1, room2, time_slot2 = gene
    print(f"Course: {course}, Lab: {is_lab}, Section: {section}, Professor: {professor}, Room1: {room1}, Slot1: {time_slot1}, Room2: {room2}, Slot2: {time_slot2}")
